In [38]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Load and preprocess the MNIST dataset
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = tf.expand_dims(x_train, axis=-1)
x_test = tf.expand_dims(x_test, axis=-1)

# Define the autoencoder model
autoencoder = models.Sequential([
    # Encoder
    layers.Input(shape=(28, 28, 1)),
    layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Conv2D(8, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2), padding='same'),
    
    # Decoder
    layers.Conv2DTranspose(8, (3, 3), activation='relu', padding='same'),
    layers.UpSampling2D((2, 2)),
    layers.Conv2DTranspose(16, (3, 3), activation='relu', padding='same'),
    layers.UpSampling2D((2, 2)),
    
    # Output layer
    layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')
])

# Compile the model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the model
autoencoder.fit(x_train, x_train, epochs=50, batch_size=128, shuffle=True, validation_data=(x_test, x_test))

# Save the model
autoencoder.save('simple_autoencoder_model.h5')

# Evaluate the model
evaluation = autoencoder.evaluate(x_test, x_test, batch_size=128)
print("Evaluation Loss:", evaluation)


Epoch 1/50
469/469 [==============================] - 4s 5ms/step - loss: 0.1512 - val_loss: 0.0902
Epoch 2/50
469/469 [==============================] - 2s 5ms/step - loss: 0.0862 - val_loss: 0.0817
Epoch 3/50
469/469 [==============================] - 2s 5ms/step - loss: 0.0808 - val_loss: 0.0785
Epoch 4/50
469/469 [==============================] - 2s 5ms/step - loss: 0.0782 - val_loss: 0.0765
Epoch 5/50
469/469 [==============================] - 2s 5ms/step - loss: 0.0766 - val_loss: 0.0752
Epoch 6/50
469/469 [==============================] - 2s 5ms/step - loss: 0.0754 - val_loss: 0.0743
Epoch 7/50
469/469 [==============================] - 2s 5ms/step - loss: 0.0745 - val_loss: 0.0734
Epoch 8/50
469/469 [==============================] - 2s 5ms/step - loss: 0.0738 - val_loss: 0.0728
Epoch 9/50
469/469 [==============================] - 2s 5ms/step - loss: 0.0733 - val_loss: 0.0723
Epoch 10/50
469/469 [==============================] - 2s 5ms/step - loss: 0.0728 - val_loss: 0.0720

In [39]:
# Display model summary
autoencoder.summary()


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_26 (Conv2D)          (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d_26 (MaxPooli  (None, 14, 14, 16)        0         
 ng2D)                                                           
                                                                 
 conv2d_27 (Conv2D)          (None, 14, 14, 8)         1160      
                                                                 
 max_pooling2d_27 (MaxPooli  (None, 7, 7, 8)           0         
 ng2D)                                                           
                                                                 
 conv2d_transpose_33 (Conv2  (None, 7, 7, 8)           584       
 DTranspose)                                                     
                                                     

In [40]:
client_model = models.Sequential(autoencoder.layers[:5])
server_model = models.Sequential(autoencoder.layers[5:])

In [41]:
client_model.compile(optimizer='adam', loss='binary_crossentropy')
server_model.compile(optimizer='adam', loss='binary_crossentropy')

In [44]:
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = tf.expand_dims(x_train, axis=-1)
x_test = tf.expand_dims(x_test, axis=-1)

# Split the dataset into two clients
x_train_client1, x_train_client2 = x_train[:30000], x_train[30000:]
x_test_client1, x_test_client2 = x_test[:5000], x_test[5000:]

In [45]:
client1_features = client_model.predict(x_train_client1)

938/938 [==============================] - 2s 2ms/step


In [46]:
client2_features = client_model.predict(x_train_client2)

938/938 [==============================] - 2s 2ms/step


In [49]:
client1_predictions = server_model.predict(client1_features)
client2_predictions = server_model.predict(client2_features)

938/938 [==============================] - 1s 2ms/step


In [50]:
evaluation_client1 = server_model.evaluate(client1_features,x_train_client1)

938/938 [==============================] - 2s 2ms/step - loss: 0.0687


In [51]:
evaluation_client2 = server_model.evaluate(client2_features,x_train_client2)

938/938 [==============================] - 2s 2ms/step - loss: 0.0694


In [55]:
test1_features = client_model.predict(x_test_client1)
test2_features = client_model.predict(x_test_client2)

157/157 [==============================] - 0s 2ms/step


In [57]:
test_eval = server_model.evaluate(test1_features,x_test_client1)
test_eval2 = server_model.evaluate(test2_features,x_test_client2)

157/157 [==============================] - 0s 2ms/step - loss: 0.0691
